# Test of shower reconstruction in simulation

This notebook explains how to access the results of reconstructed showers in the simulation with Random Forest.

The results are from the test sample (the training sample is used for training the classifier)

First, importing the required modules

In [1]:
import numpy as np
import pandas as pd
import ROOT as r
import root_numpy as rp

Welcome to JupyROOT 6.22/06


## Opening input dataframes
The classifier separates between 0 (background) and 1 (signal) classes
There are three dataframes we are interested in.

1. **Classification Report** of the classifier

2. **Confusion matrix**

3. **Predicition.csv** (contains the reconstructed showers)


In [2]:
nrun = 6
inputpath = "/eos/experiment/ship/data/DESY19TB/DE19_R{}/RandomForest/".format(nrun)
outputfilename = "plots/DESYRUN{}histos_sim.root".format(nrun) #create an empty plots folder the first time
!ls $inputpath

Classification_Report1.csv  Prediction.csv	      Report_Forest1.txt
Confusion_matrix1.csv	    RandomForest_histos.root  Result_data.csv


In [3]:
outputfile = r.TFile(outputfilename,"RECREATE")

### Classification Report

it describes the performance of our classifier. This is usually the most important, since we use it to study how the classifier behaves in separating signal and background in our simulation.

The performances are evaluated as

* Precision P:

$ P = \frac{TP}{TP + FN} $

* Recall R:

$ R = \frac{TP}{TP + FN} $

* F_score:

$ F = \frac{2 * R * P } {R + P} $

In [4]:
class_report = pd.read_csv(inputpath+"Classification_Report1.csv")
class_report

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.971128,0.995138,0.982986,587576.000000
1,1,0.883890,0.555771,0.682439,39133.000000
2,accuracy,0.967703,0.967703,0.967703,0.967703
3,macro avg,0.927509,0.775454,0.832713,626709.000000
4,weighted avg,0.965681,0.967703,0.964219,626709.000000


### Confusion matrix
it describes how a true signal (1) is recognized as a true signal (1->1) or a true background (1->0).

At the same time 0->0 is background reconstructed as background, while 0->1 is background mistaken as signal

In [5]:
conf_matrix = pd.read_csv(inputpath+"Confusion_matrix1.csv")
conf_matrix

,Unnamed: 0,0,1
0,0,584719,2857
1,1,17384,21749


## Prediction csv
It contains all the reconstructed showers from the test sample.

In [6]:
showerdataframe = pd.read_csv(inputpath+"Prediction.csv")
showerdataframe.head()

,Unnamed: 0,ID,x,y,z,TX,TY,PID,X_Next,Y_Next,...,dTY,dR,dT,DeltaT,Par_impact_nor,Angolo_cono,MCEvent,Ishower,Y_test,Y_pred_forest
0,0,3,41103.620,40302.990,-32875.0,0.003137,-0.001757,23,41105.682641,40301.835026,...,-0.187349,364.224136,0.233803,0.001462,0.005620,0.002687,0,0.0,1,1
1,1,137,41174.703,40307.586,-32875.0,0.054414,0.017520,23,41210.480227,40319.105298,...,-0.206626,336.007073,0.223918,0.054969,0.086328,0.029612,0,0.0,1,1
2,2,156,41142.895,40322.848,-32875.0,0.043573,0.036087,23,41171.544472,40346.575356,...,-0.225193,362.684683,0.244178,0.054383,0.075617,0.019419,0,0.0,1,1
3,3,251626,41469.242,40109.670,-30245.0,0.186545,-0.003624,23,41591.895423,40107.287133,...,-0.185482,205.870496,0.186810,0.182326,0.260341,0.079456,-999,0.0,0,1
4,4,901283,40714.340,40624.617,-30245.0,0.035851,-0.069336,23,40737.911833,40579.028619,...,-0.119770,858.596000,0.164085,0.075765,0.037715,0.095419,-999,0.0,0,0


### Signal extraction, size of showers
let us extract only the component 1->1 (signal correctly recognized as signal).

How many segments are present for shower?

In [7]:
def extractsignalshower(datadf):
 #removing na
 df = datadf[datadf["DeltaT"].isna()==False] 
 df = df[df["Y_pred_forest"]==1]
 signaldf = df[df["Y_test"]==1]

 #number of segments witin the same Ishower
 sizedataset = signaldf.groupby("Ishower").count()
 size = sizedataset["ID"].to_numpy()
 return signaldf, size

In [8]:
signaldf, size = extractsignalshower(showerdataframe)

In [9]:
signaldf.head()

,Unnamed: 0,ID,x,y,z,TX,TY,PID,X_Next,Y_Next,...,dTY,dR,dT,DeltaT,Par_impact_nor,Angolo_cono,MCEvent,Ishower,Y_test,Y_pred_forest
0,0,3,41103.620,40302.990,-32875.0,0.003137,-0.001757,23,41105.682641,40301.835026,...,-0.187349,364.224136,0.233803,0.001462,0.005620,0.002687,0,0.0,1,1
1,1,137,41174.703,40307.586,-32875.0,0.054414,0.017520,23,41210.480227,40319.105298,...,-0.206626,336.007073,0.223918,0.054969,0.086328,0.029612,0,0.0,1,1
2,2,156,41142.895,40322.848,-32875.0,0.043573,0.036087,23,41171.544472,40346.575356,...,-0.225193,362.684683,0.244178,0.054383,0.075617,0.019419,0,0.0,1,1
6,6,4,41109.887,40304.770,-31560.0,0.001169,-0.000275,22,41110.655858,40304.588962,...,-0.001481,5.127446,0.002463,-0.000932,0.004483,0.003433,0,0.0,1,1
7,7,138,41204.910,40316.348,-31560.0,0.035047,0.007087,22,41227.953683,40321.007818,...,0.010433,6.215314,0.021994,0.033608,0.063354,0.027616,0,0.0,1,1


## Plotting size histograms
Let us check the size of reconstructed showers

In [10]:
#make histograms
hsizeML = r.TH1D("hsizeML","Size of showers reconstructed by Random Forest in test simulation;Nsegments", 20,0,200)
rp.fill_hist(hsizeML,size)

In [11]:
%jsroot on
c = r.TCanvas()
hsizeML.Draw()
c.Draw()

## Making a selection on the shower
We accept usually showers only with a mininmum of associated segments

In [12]:
def acceptshower(signaldf,showersize, minimumsize):
    goodshowers = np.where(showersize >= minimumsize)[0] #it returns an array of array, for some reason, we want the actual numbers
    print(goodshowers[1])
    gooddf = signaldf[signaldf["Ishower"].isin(goodshowers)]
    return gooddf

In [13]:
minimumsize = 50
gooddataframe = acceptshower(signaldf,size,minimumsize)

1


In [14]:
hIPnorm = r.TH1D("hIPnorm","Impact parameter over distance along axis in test simulation;IP/#DeltaZ",30,0.,0.3)
hthetaprime = r.TH1D("hthetaprime","Cone angle with respect to shower start for RUN{};#theta'[rad]",40,0,0.04)

rp.fill_hist(hIPnorm, gooddataframe["Par_impact_nor"].to_numpy())
rp.fill_hist(hthetaprime,gooddataframe["Angolo_cono"].to_numpy())

In [15]:
cIP = r.TCanvas()
hIPnorm.Draw()
cIP.Draw()

In [16]:
cTheta = r.TCanvas()
hthetaprime.Draw()
cTheta.Draw()

## Energy resolution
We now estimate the energy from the number of associated segments. Calibration parameters have been computed from Monte Carlo.

In [17]:
import Energymeasurement as Emeas

In [23]:
goodsize = size[size>minimumsize]
Erec = Emeas.estimate_energy(size) #replace with goodsize to use only "good showers"

SigmaE = (Erec - 4.)/4.

Let us draw the histogram of the resolution

In [24]:
cres = r.TCanvas()
hres = r.TH1D('hres', '#DeltaE/E distribution; #DeltaE/E;Entries', 10, -1, 1)

rp.fill_hist(hres,SigmaE)

hres.Draw()
cres.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: hres (Potential memory leak).


## Saving histograms into ROOT file
We save the obtained histograms into an output file to access them in later codes

In [19]:
outputfile.cd()
hsizeML.Write()
hIPnorm.Write()
hthetaprime.Write()

897

In [20]:
outputfile.Close()